In [1]:
# fuel_shock_atf_realtime.py
# Purpose: Fetch real ATF time series (India) and run the fuel-shock + UDAN scenario analysis
# Uses OpenFlights for route universe, tries data.gov.in and IOCL for ATF series, falls back to FRED.
# Produces CSVs, charts, and an executive summary.
# Problem statement (IIT Guwahati Winter Consulting, Case 1) used to structure deliverables. :contentReference[oaicite:1]{index=1}

import os
import io
import math
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pandas_datareader.data import DataReader
from bs4 import BeautifulSoup

# ---------------------------
# OUTPUT
# ---------------------------
OUTDIR = "outputs"
os.makedirs(OUTDIR, exist_ok=True)

# ---------------------------
# ASSUMPTIONS (editable)
# ---------------------------
ASSUMPTIONS = {
    # fleet / pax
    "avg_seats": 78,
    "default_load_factor": 0.72,
    "avg_fare_inr": 3000.0,
    "avg_ancillary_inr": 400.0,
    # costs
    "fixed_cost_per_flight_inr": 250000.0,
    "liters_per_km_per_seat": 0.03,
    # currency
    "usd_to_inr": 82.0,
    # fuel shock scenario (if you want to shock relative to latest)
    "fuel_shock_pct": 0.25,
    # pricing scenario (not primary here, we focus on Option B)
    "fare_increase_pct": 0.06,
    "price_elasticity": -1.0,
    # UDAN/support scenario (Option B)
    "udan_monthly_support_inr": 50000.0,
    "flights_per_month_default": 60,
    # OpenFlights source (route universe)
    "openflights_airports_url": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",
    "openflights_routes_url": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat",
    # data.gov.in ATF resource (we'll try to scrape CSV link here)
    "data_gov_atf_resource_url": "https://www.data.gov.in/resource/effective-date-wise-prices-aviation-turbine-fuel-atf-indian-oil-corporation-limited-iocl",
    # IOCL pages to try if data.gov.in fails
    "iocl_atf_pages": [
        "https://iocl.com/aviation-fuel",
        "https://iocl.com/atf-domestic-airlines",
        "https://iocl.com/prices-of-petroleum-products"
    ],
    # FRED fallback series (US Gulf Coast jet fuel) - used only if India sources fail
    "fred_series": "DJFUELUSGULF"
}

# ---------------------------
# UTILITIES
# ---------------------------
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2.0)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2.0)**2
    return 2 * R * math.asin(math.sqrt(a))

def safe_get(url, timeout=20):
    try:
        r = requests.get(url, timeout=timeout)
        r.raise_for_status()
        return r
    except Exception as e:
        print(f"  [WARN] Failed to GET {url}: {e}")
        return None

# ---------------------------
# STEP 1: Build route universe with OpenFlights
# ---------------------------
print("STEP 1: Build route universe (OpenFlights)...")
airports_txt = requests.get(ASSUMPTIONS["openflights_airports_url"]).text
routes_txt = requests.get(ASSUMPTIONS["openflights_routes_url"]).text

airports_cols = ["id","name","city","country","iata","icao","lat","lon","alt","tz","dst","tzdb","type","src"]
routes_cols = ["airline","airline_id","src","src_id","dst","dst_id","codeshare","stops","equip"]

airports = pd.read_csv(io.StringIO(airports_txt), header=None, names=airports_cols, dtype=str, keep_default_na=False)
routes = pd.read_csv(io.StringIO(routes_txt), header=None, names=routes_cols, dtype=str, keep_default_na=False)

# sanitize numeric fields
airports["lat"] = pd.to_numeric(airports["lat"], errors="coerce")
airports["lon"] = pd.to_numeric(airports["lon"], errors="coerce")
airports["id"] = pd.to_numeric(airports["id"], errors="coerce")
routes["src_id"] = pd.to_numeric(routes["src_id"], errors="coerce")
routes["dst_id"] = pd.to_numeric(routes["dst_id"], errors="coerce")

india_airports = airports[airports["country"].str.strip().str.lower() == "india"].copy()
india_airport_ids = set(india_airports["id"].dropna().astype(int).tolist())

routes["in_india"] = routes.apply(
    lambda r: (pd.notna(r["src_id"]) and pd.notna(r["dst_id"]) and int(r["src_id"]) in india_airport_ids and int(r["dst_id"]) in india_airport_ids),
    axis=1
)
india_routes = routes[routes["in_india"]].copy().reset_index(drop=True)
print(f" - Found {len(india_airports)} India airports and {len(india_routes)} intra-India routes (raw).")

id2airport = india_airports.set_index("id")[["iata","city","lat","lon","name"]]
enriched = []
for _, r in india_routes.iterrows():
    try:
        s = id2airport.loc[int(r["src_id"])]
        d = id2airport.loc[int(r["dst_id"])]
    except Exception:
        continue
    if pd.isna(s["lat"]) or pd.isna(d["lat"]):
        continue
    dist = haversine_km(float(s["lat"]), float(s["lon"]), float(d["lat"]), float(d["lon"]))
    enriched.append({
        "src_iata": s["iata"],
        "dst_iata": d["iata"],
        "src_city": s["city"],
        "dst_city": d["city"],
        "distance_km": round(dist,1)
    })

routes_df = pd.DataFrame(enriched).drop_duplicates().reset_index(drop=True)
if routes_df.empty:
    # fallback small synthetic sample
    routes_df = pd.DataFrame([
        {"src_iata":"GAU","dst_iata":"IMF","src_city":"Guwahati","dst_city":"Imphal","distance_km":350},
        {"src_iata":"DEL","dst_iata":"JAI","src_city":"Delhi","dst_city":"Jaipur","distance_km":260},
        {"src_iata":"PAT","dst_iata":"CCU","src_city":"Patna","dst_city":"Kolkata","distance_km":240}
    ])
    print(" - Warning: route universe empty; using small synthetic sample (demo).")
else:
    print(f" - Unique intra-India route pairs after dedupe: {len(routes_df)}")

# ---------------------------
# STEP 2: Try fetching real ATF time series (India)
# Strategy:
#  1) Try to find a CSV on the data.gov.in resource page
#  2) If not found, scrape IOCL pages for ATF price tables
#  3) Fallback: use FRED jet fuel series converted to INR
# ---------------------------
print("\nSTEP 2: Fetching India ATF time series (data.gov.in -> IOCL -> FRED fallback)...")

def try_data_gov_csv(resource_page_url):
    print("  - Trying data.gov.in resource page for CSV link...")
    r = safe_get(resource_page_url)
    if r is None:
        return None
    soup = BeautifulSoup(r.text, "lxml")
    # find links that look like CSV or download endpoints
    candidates = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.lower().endswith(".csv"):
            candidates.append(href)
        # sometimes 'download' links are present with /download or /download? etc
        if "download" in href.lower() and (href.lower().endswith(".csv") or "format=csv" in href.lower()):
            candidates.append(href)
    # normalize and attempt download
    for href in candidates:
        if href.startswith("//"):
            href = "https:" + href
        if href.startswith("/"):
            href = "https://data.gov.in" + href
        print(f"    trying candidate CSV: {href}")
        rr = safe_get(href)
        if rr is None:
            continue
        try:
            df = pd.read_csv(io.StringIO(rr.text))
            print("    -> Successfully loaded CSV from data.gov.in resource.")
            return df
        except Exception as e:
            # Try saving to bytes and read by pandas
            try:
                df = pd.read_csv(io.BytesIO(rr.content))
                print("    -> Successfully loaded CSV (bytes) from data.gov.in resource.")
                return df
            except Exception:
                print(f"    -> Failed to parse candidate {href}: {e}")
                continue
    print("  - No usable CSV found on data.gov.in page.")
    return None

def try_iocl_scrape(pages):
    print("  - Trying IOCL pages for ATF price tables...")
    for url in pages:
        print(f"    trying {url}")
        r = safe_get(url)
        if r is None:
            continue
        try:
            tables = pd.read_html(r.text)
        except Exception as e:
            # sometimes HTML parsing fails; try BeautifulSoup to find table-like content
            tables = []
        for t in tables:
            # look for columns or values that indicate ATF or 'ATF' string
            text = " ".join(t.columns.astype(str)).lower()
            if any("atf" in c.lower() or "turbine" in c.lower() or "aviation" in c.lower() for c in t.columns.astype(str)) or t.shape[1] <= 3:
                # attempt to coerce to time series: find date-like column and price-like column
                # heuristics: columns with 'date', 'effective', 'month' OR numeric series with 'price'
                df = t.copy()
                # simple cleaning: try to melt if first col are dates
                # return df for manual inspection - we'll let caller process
                print(f"    -> Found a candidate table on {url} with shape {df.shape}")
                return df
    print("  - No usable table found on IOCL pages.")
    return None

atf_df = None
# 1) data.gov.in attempt
atf_df = try_data_gov_csv(ASSUMPTIONS["data_gov_atf_resource_url"])

# 2) IOCL attempt
if atf_df is None:
    iocl_candidate = try_iocl_scrape(ASSUMPTIONS["iocl_atf_pages"])
    if iocl_candidate is not None:
        # heuristic: try to reshape candidate into time series if possible
        atf_df = iocl_candidate

# 3) fallback to FRED jet fuel converted to INR
if atf_df is None:
    print("  - Falling back to FRED jet fuel series (converted to INR).")
    try:
        end = datetime.today()
        start = datetime(end.year - 5, end.month, end.day)
        fred_series = DataReader(ASSUMPTIONS["fred_series"], "fred", start, end)
        fred_series = fred_series.dropna().reset_index()
        fred_series.columns = ["date", "usd_per_gallon"]
        fred_series["usd_per_liter"] = fred_series["usd_per_gallon"] / 3.78541
        fred_series["inr_per_liter"] = fred_series["usd_per_liter"] * ASSUMPTIONS["usd_to_inr"]
        atf_df = fred_series[["date","inr_per_liter"]].rename(columns={"inr_per_liter":"price_inr_per_liter","date":"date"})
        print("  - FRED fallback loaded.")
    except Exception as e:
        print("  - FRED fetch failed:", e)
        # final fallback: synthetic price series
        dates = pd.date_range(end=datetime.today(), periods=24, freq='M')
        prices = np.linspace(110, 160, len(dates))  # INR/L arbitrary
        atf_df = pd.DataFrame({"date":dates, "price_inr_per_liter":prices})
        print("  - Using synthetic ATF series (last-resort).")

# ---------------------------
# STEP 2b: Normalize ATF DataFrame into a time series: columns ['date','price_inr_per_liter']
# ---------------------------
print("\nNormalizing ATF time series...")

def normalize_atf(df):
    # Attempt various heuristics depending on the source structure
    df = df.copy()
    # If column names contain 'price' or 'inr' or '₹' or 'Rs', try to find price col
    text_cols = [c.lower() for c in df.columns.astype(str)]
    # find date col
    date_col = None
    price_col = None
    for c in df.columns:
        lc = str(c).lower()
        if any(k in lc for k in ["date","effective","month","period"]):
            date_col = c
        if any(k in lc for k in ["price","rate","inr","₹","rs","amount","price_inr","price_kl"]):
            price_col = c
    # If not found, try positional heuristics
    if date_col is None:
        # try first column if it looks like dates
        try:
            tmp = pd.to_datetime(df.iloc[:,0], errors='coerce')
            if tmp.notna().sum() > 0:
                date_col = df.columns[0]
        except Exception:
            pass
    if price_col is None:
        # try numeric columns that have biggest variance
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if numeric_cols:
            # pick the numeric column with the largest mean (likely price in INR per KL if huge)
            price_col = numeric_cols[0]
    # If still not found, attempt to melt table: some tables have months as columns
    if date_col is None or price_col is None:
        # try to find date in index or column headers (like month columns)
        # Try simple wide->long transformation if first col is month names and remaining are values
        try:
            # assume first col is 'month' or index-like
            if df.shape[1] >= 2:
                # convert header to string and check if any header contains year-like text
                header_text = " ".join([str(c) for c in df.columns])
                # try melt
                melted = df.melt(id_vars=[df.columns[0]], var_name="period", value_name="value")
                # try parse date from 'period' or first col
                for col in [melted.columns[0], "period"]:
                    try:
                        candidate = pd.to_datetime(melted[col], errors='coerce')
                        if candidate.notna().sum() > 0:
                            melted["date"] = candidate
                            melted["price"] = pd.to_numeric(melted["value"], errors='coerce')
                            result = melted[["date","price"]].dropna().drop_duplicates()
                            result = result.rename(columns={"price":"price_inr_per_liter"})
                            return result
                    except Exception:
                        continue
        except Exception:
            pass
    # If we have both columns, coerce and return
    if date_col is not None and price_col is not None:
        try:
            result = pd.DataFrame()
            result["date"] = pd.to_datetime(df[date_col], errors='coerce')
            # price could be per KL (e.g., INR per KL). We need to detect scale:
            price_raw = pd.to_numeric(df[price_col].astype(str).str.replace(",","").str.replace("₹","").str.replace("Rs","", case=False), errors='coerce')
            # Heuristic: if mean price > 1000, likely INR per KL -> convert to per L
            mean_price = price_raw.mean(skipna=True)
            if mean_price is not None and mean_price > 1000:
                # assume per kiloliter -> divide by 1000
                result["price_inr_per_liter"] = price_raw / 1000.0
            else:
                result["price_inr_per_liter"] = price_raw
            result = result.dropna(subset=["date","price_inr_per_liter"])
            result = result.sort_values("date").reset_index(drop=True)
            return result
        except Exception as e:
            print("  [WARN] normalize_atf failed to coerce date/price columns:", e)
    # final fallback: try numeric column + index as dates
    tmp = df.select_dtypes(include=[np.number])
    if not tmp.empty:
        result = pd.DataFrame()
        # create fake monthly dates
        n = tmp.shape[0]
        result["date"] = pd.date_range(end=datetime.today(), periods=n, freq="M")
        result["price_inr_per_liter"] = tmp.iloc[:,0].astype(float).values
        return result
    # If nothing worked, return None
    return None

atf_ts = normalize_atf(atf_df)
if atf_ts is None or atf_ts.empty:
    print("  [ERROR] Could not normalize ATF series. Creating fallback synthetic series.")
    dates = pd.date_range(end=datetime.today(), periods=36, freq='M')
    prices = np.linspace(100, 140, len(dates))
    atf_ts = pd.DataFrame({"date":dates,"price_inr_per_liter":prices})

# Save ATF time series
atf_ts.to_csv(os.path.join(OUTDIR, "atf_time_series.csv"), index=False, encoding="utf-8")
print(f"  - ATF time series saved to outputs/atf_time_series.csv (rows: {len(atf_ts)})")

# plot ATF series
plt.figure(figsize=(9,4))
plt.plot(atf_ts["date"], atf_ts["price_inr_per_liter"], marker="o")
plt.xlabel("Date")
plt.ylabel("ATF price (INR / L)")
plt.title("ATF price time series (India) — derived from data.gov.in / IOCL / fallback")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, "atf_timeseries.png"))
plt.close()
print("  - ATF time series plot saved to outputs/atf_timeseries.png")

# ---------------------------
# STEP 3: Build route-level P&L using latest ATF price
# ---------------------------
print("\nSTEP 3: Building route-level economics using latest ATF price...")

latest_price_inr_per_liter = float(atf_ts["price_inr_per_liter"].dropna().iloc[-1])
print(f"  - Latest ATF (INR/L) used: {latest_price_inr_per_liter:.2f} INR/L")

df = routes_df.copy()
df["seats"] = ASSUMPTIONS["avg_seats"]
df["load_factor"] = ASSUMPTIONS["default_load_factor"]
df["pax_onboard"] = (df["seats"] * df["load_factor"]).round().astype(int)
df["avg_fare_inr"] = ASSUMPTIONS["avg_fare_inr"]
df["ancillary_inr"] = ASSUMPTIONS["avg_ancillary_inr"]
df["revenue_per_flight"] = df["pax_onboard"] * (df["avg_fare_inr"] + df["ancillary_inr"])

df["fuel_liters_per_flight"] = (df["distance_km"] * df["seats"] * ASSUMPTIONS["liters_per_km_per_seat"]).round(2)
df["fuel_cost_inr"] = df["fuel_liters_per_flight"] * latest_price_inr_per_liter
df["fixed_cost_inr"] = ASSUMPTIONS["fixed_cost_per_flight_inr"]
df["total_cost_inr"] = df["fuel_cost_inr"] + df["fixed_cost_inr"]
df["margin_per_flight"] = df["revenue_per_flight"] - df["total_cost_inr"]
df["margin_per_pax"] = df["margin_per_flight"] / df["pax_onboard"].replace(0, np.nan)

base_csv = os.path.join(OUTDIR, "route_economics_base.csv")
df.to_csv(base_csv, index=False, encoding="utf-8")
print(f"  - Saved base P&L to {base_csv}")

# ---------------------------
# STEP 4: Fuel shock + Option B (UDAN) scenario
# ---------------------------
print("\nSTEP 4: Simulating fuel shock and Option B (UDAN support) scenario...")

# Fuel shock - relative to latest price
df_shock = df.copy()
df_shock["fuel_cost_inr_shock"] = df_shock["fuel_liters_per_flight"] * (latest_price_inr_per_liter * (1.0 + ASSUMPTIONS["fuel_shock_pct"]))
df_shock["total_cost_inr_shock"] = df_shock["fuel_cost_inr_shock"] + df_shock["fixed_cost_inr"]
df_shock["margin_post_shock"] = df_shock["revenue_per_flight"] - df_shock["total_cost_inr_shock"]

# Option B: UDAN-like support (per-month support -> per-flight)
support_month = ASSUMPTIONS["udan_monthly_support_inr"]
flights_per_month = ASSUMPTIONS["flights_per_month_default"]
support_per_flight = support_month / flights_per_month
df_shock["margin_with_udan"] = df_shock["margin_post_shock"] + support_per_flight

postshock_csv = os.path.join(OUTDIR, "route_economics_postshock_udan.csv")
df_shock.to_csv(postshock_csv, index=False, encoding="utf-8")
print(f"  - Saved post-shock + UDAN scenario to {postshock_csv}")

# ---------------------------
# STEP 5: KPI summary focused on Option B
# ---------------------------
print("\nSTEP 5: KPI summary and UDAN impact...")

total_routes = len(df_shock)
num_loss_base = (df_shock["margin_per_flight"] < 0).sum()
num_loss_post = (df_shock["margin_post_shock"] < 0).sum()
num_saved_by_udan = ((df_shock["margin_post_shock"] < 0) & (df_shock["margin_with_udan"] >= 0)).sum()
num_still_negative_with_udan = (df_shock["margin_with_udan"] < 0).sum()

avg_margin_base = df_shock["margin_per_flight"].mean()
avg_margin_post = df_shock["margin_post_shock"].mean()
avg_margin_udan = df_shock["margin_with_udan"].mean()

summary = [
    f"Analysis timestamp (UTC): {datetime.utcnow().isoformat()}",
    "Problem statement: IIT Guwahati Winter Consulting — Case 1. :contentReference[oaicite:2]{index=2}",
    "",
    f"Total routes analyzed: {total_routes}",
    f"Routes loss-making (base): {num_loss_base}",
    f"Routes loss-making (post fuel shock +{ASSUMPTIONS['fuel_shock_pct']*100:.0f}%): {num_loss_post}",
    f"Routes that UDAN support (₹{support_month:.0f}/month) can save (flip to non-loss): {num_saved_by_udan}",
    f"Routes still loss-making even with UDAN: {num_still_negative_with_udan}",
    "",
    f"Avg margin per flight (base): {avg_margin_base:,.0f} INR",
    f"Avg margin per flight (post-shock): {avg_margin_post:,.0f} INR",
    f"Avg margin per flight (with UDAN): {avg_margin_udan:,.0f} INR",
    "",
    "Key assumptions:",
    f" - Latest ATF used: {latest_price_inr_per_liter:.2f} INR/L (derived from atf_time_series).",
    f" - Seats/load factor: {ASSUMPTIONS['avg_seats']}/{ASSUMPTIONS['default_load_factor']}",
    f" - Fixed cost per flight: ₹{ASSUMPTIONS['fixed_cost_per_flight_inr']:.0f}",
    f" - UDAN monthly support assumed: ₹{support_month:.0f} (→ ₹{support_per_flight:.0f}/flight at {flights_per_month} flights/mo)",
    "",
    "Recommendation (from analysis):",
    " - Use UDAN selectively to stabilize socially-important, high-volatility regional routes (Option B as buffer).",
    " - Primary action remains network optimization & selective pricing on core routes (Option A) — UDAN supports downside protection.",
    ""
]
summary_txt = "\n".join(summary)
with open(os.path.join(OUTDIR, "summary_metrics_udan.txt"), "w", encoding="utf-8") as f:
    f.write(summary_txt)
print("  - Summary saved to outputs/summary_metrics_udan.txt")

# ---------------------------
# STEP 6: Charts for slides (ATF series, margins, UDAN impact)
# ---------------------------
print("\nSTEP 6: Producing charts (ATF series, margins distribution, UDAN impact)...")

# ATF time series plot already saved earlier as atf_timeseries.png; ensure exists by re-plotting simply
plt.figure(figsize=(9,4))
plt.plot(atf_ts["date"], atf_ts["price_inr_per_liter"], marker="o")
plt.xlabel("Date")
plt.ylabel("ATF price (INR / L)")
plt.title("ATF price (India) — time series (data.gov.in / IOCL / fallback)")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, "atf_timeseries.png"))
plt.close()

# margins distribution base vs post-shock vs with UDAN
plt.figure(figsize=(10,6))
plt.hist(df_shock["margin_per_flight"], bins=60, alpha=0.5, label="Base margin")
plt.hist(df_shock["margin_post_shock"], bins=60, alpha=0.5, label="Post-shock margin")
plt.hist(df_shock["margin_with_udan"], bins=60, alpha=0.5, label="With UDAN margin")
plt.axvline(0, color="k", linewidth=0.8)
plt.xlabel("Margin per flight (INR)")
plt.ylabel("Count")
plt.title("Margin distribution: base vs post-shock vs with UDAN")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, "margins_udan_comparison.png"))
plt.close()

# UDAN impact: how many routes recovered per city-pair (top)
recovered = df_shock[(df_shock["margin_post_shock"] < 0) & (df_shock["margin_with_udan"] >= 0)].copy()
recovered = recovered.assign(margin_deficit = df_shock["margin_post_shock"] * -1).sort_values("margin_post_shock")
recovered.head(20).to_csv(os.path.join(OUTDIR, "routes_saved_by_udan_top20.csv"), index=False, encoding="utf-8")

print("  - Charts saved: outputs/atf_timeseries.png, outputs/margins_udan_comparison.png")
print("  - routes saved by UDAN (top20) saved to outputs/routes_saved_by_udan_top20.csv")

# ---------------------------
# STEP 7: Quick PPTX (5 slides) including ATF timeseries & UDAN bullets (optional)
# ---------------------------
try:
    from pptx import Presentation
    from pptx.util import Inches
    prs = Presentation()
    # Title slide
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    slide.shapes.title.text = "Regional Airline — Fuel Shock & UDAN Analysis"
    slide.placeholders[1].text = "Diagnosis, UDAN scenario & recommendation (IIT Guwahati Winter Consulting)"
    # Slide 1: ATF timeseries
    s1 = prs.slides.add_slide(prs.slide_layouts[5])
    s1.shapes.title.text = "Slide 1 — ATF time series (India)"
    s1.shapes.add_picture(os.path.join(OUTDIR, "atf_timeseries.png"), Inches(0.5), Inches(1.25), width=Inches(9))
    # Slide 2: Margin impact
    s2 = prs.slides.add_slide(prs.slide_layouts[5])
    s2.shapes.title.text = "Slide 2 — Margin impact: base vs post-shock vs UDAN"
    s2.shapes.add_picture(os.path.join(OUTDIR, "margins_udan_comparison.png"), Inches(0.5), Inches(1.25), width=Inches(9))
    # Slide 3: Vulnerable routes (top 6)
    s3 = prs.slides.add_slide(prs.slide_layouts[5])
    s3.shapes.title.text = "Slide 3 — Vulnerable routes (post-shock)"
    vuln = df_shock.sort_values("margin_post_shock").head(6)
    tx = s3.shapes.add_textbox(Inches(0.5), Inches(1.0), Inches(9), Inches(4))
    tf = tx.text_frame
    for _, row in vuln.iterrows():
        tf.add_paragraph().text = f"{row['src_iata']}-{row['dst_iata']} | dist {row['distance_km']} km | post_shock margin {row['margin_post_shock']:.0f} INR"
    # Slide 4: UDAN impact & recommendation
    s4 = prs.slides.add_slide(prs.slide_layouts[1])
    s4.shapes.title.text = "Slide 4 — UDAN impact & Recommendation"
    s4.placeholders[1].text = (
        "Option B (UDAN) can stabilize a subset of fragile regional routes short-term.\n"
        f"It saves {num_saved_by_udan} routes from being loss-making under a +{ASSUMPTIONS['fuel_shock_pct']*100:.0f}% fuel shock.\n"
        "Recommendation: Use UDAN selectively as downside insurance; prioritize network optimization on core routes."
    )
    # Slide 5: KPIs & fallback plan
    s5 = prs.slides.add_slide(prs.slide_layouts[1])
    s5.shapes.title.text = "Slide 5 — KPIs & Exit Triggers"
    s5.placeholders[1].text = (
        "KPIs: Route-level EBITDA margin, CASK, Cash burn rate.\n"
        "Triggers: Exit route if margin < threshold for 2 consecutive quarters.\n"
        "Begin structural redesign planning (fleet mix) as Phase 2; do not lock entire fleet into UDAN."
    )
    pptx_path = os.path.join(OUTDIR, "fuel_shock_udan_deck.pptx")
    prs.save(pptx_path)
    print("  - PPTX generated: outputs/fuel_shock_udan_deck.pptx")
except Exception as e:
    print("  - PPTX generation skipped (python-pptx may be missing or failed):", e)

print("\nDONE. Outputs are in the 'outputs' folder:")
print(" - outputs/atf_time_series.csv")
print(" - outputs/atf_timeseries.png")
print(" - outputs/route_economics_base.csv")
print(" - outputs/route_economics_postshock_udan.csv")
print(" - outputs/routes_saved_by_udan_top20.csv")
print(" - outputs/summary_metrics_udan.txt")
print(" - outputs/margins_udan_comparison.png")
print(" - optional: outputs/fuel_shock_udan_deck.pptx (if python-pptx installed)")

# End of script


STEP 1: Build route universe (OpenFlights)...
 - Found 148 India airports and 955 intra-India routes (raw).
 - Unique intra-India route pairs after dedupe: 343

STEP 2: Fetching India ATF time series (data.gov.in -> IOCL -> FRED fallback)...
  - Trying data.gov.in resource page for CSV link...
  - No usable CSV found on data.gov.in page.
  - Trying IOCL pages for ATF price tables...
    trying https://iocl.com/aviation-fuel


C:\Users\Anklesh\AppData\Local\Temp\ipykernel_25108\3829118997.py:201: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


    trying https://iocl.com/atf-domestic-airlines
    trying https://iocl.com/prices-of-petroleum-products
  - No usable table found on IOCL pages.
  - Falling back to FRED jet fuel series (converted to INR).
  - FRED fallback loaded.

Normalizing ATF time series...
  - ATF time series saved to outputs/atf_time_series.csv (rows: 1245)
  - ATF time series plot saved to outputs/atf_timeseries.png

STEP 3: Building route-level economics using latest ATF price...
  - Latest ATF (INR/L) used: 41.35 INR/L
  - Saved base P&L to outputs\route_economics_base.csv

STEP 4: Simulating fuel shock and Option B (UDAN support) scenario...
  - Saved post-shock + UDAN scenario to outputs\route_economics_postshock_udan.csv

STEP 5: KPI summary and UDAN impact...
  - Summary saved to outputs/summary_metrics_udan.txt

STEP 6: Producing charts (ATF series, margins distribution, UDAN impact)...


C:\Users\Anklesh\AppData\Local\Temp\ipykernel_25108\3829118997.py:428: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f"Analysis timestamp (UTC): {datetime.utcnow().isoformat()}",


  - Charts saved: outputs/atf_timeseries.png, outputs/margins_udan_comparison.png
  - routes saved by UDAN (top20) saved to outputs/routes_saved_by_udan_top20.csv
  - PPTX generated: outputs/fuel_shock_udan_deck.pptx

DONE. Outputs are in the 'outputs' folder:
 - outputs/atf_time_series.csv
 - outputs/atf_timeseries.png
 - outputs/route_economics_base.csv
 - outputs/route_economics_postshock_udan.csv
 - outputs/routes_saved_by_udan_top20.csv
 - outputs/summary_metrics_udan.txt
 - outputs/margins_udan_comparison.png
 - optional: outputs/fuel_shock_udan_deck.pptx (if python-pptx installed)


In [2]:
pip install pandas numpy matplotlib requests pandas_datareader beautifulsoup4 lxml pulp

Note: you may need to restart the kernel to use updated packages.


In [3]:
# ==========================================================
# MBB-STYLE ATF SHOCK + UDAN OPTIMIZATION (A+B+C+D)
# ==========================================================

import os, io, math, requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pandas_datareader.data import DataReader
from bs4 import BeautifulSoup
import pulp

# ---------------- CONFIG ----------------
OUTDIR = "outputs"
os.makedirs(OUTDIR, exist_ok=True)

ASSUMPTIONS = {
    "avg_seats": 78,
    "load_factor": 0.72,
    "avg_fare": 3000,
    "ancillary": 400,
    "fixed_cost": 250000,
    "fuel_liters_km_seat": 0.03,
    "usd_to_inr": 82,
    "flights_per_month": 60,
    "horizon_months": 12,
    "fuel_shocks": [0.15, 0.25, 0.40],
    "budgets": [5e6, 10e6, 20e6, 40e6],  # INR
    "openflights_airports": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",
    "openflights_routes": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat",
    "fred_series": "DJFUELUSGULF"
}

# ---------------- HELPERS ----------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dp = math.radians(lat2 - lat1)
    dl = math.radians(lon2 - lon1)
    a = math.sin(dp/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dl/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# ---------------- ROUTES ----------------
airports = pd.read_csv(ASSUMPTIONS["openflights_airports"], header=None)
routes = pd.read_csv(ASSUMPTIONS["openflights_routes"], header=None)

airports.columns = ["id","name","city","country","iata","icao","lat","lon","alt","tz","dst","tzdb","type","src"]
routes.columns = ["airline","airline_id","src","src_id","dst","dst_id","codeshare","stops","equip"]

airports["lat"] = pd.to_numeric(airports["lat"], errors="coerce")
airports["lon"] = pd.to_numeric(airports["lon"], errors="coerce")
airports["id"] = pd.to_numeric(airports["id"], errors="coerce")
routes["src_id"] = pd.to_numeric(routes["src_id"], errors="coerce")
routes["dst_id"] = pd.to_numeric(routes["dst_id"], errors="coerce")

india = airports[airports["country"].str.lower() == "india"]
ids = set(india["id"].dropna().astype(int))
routes = routes[routes["src_id"].isin(ids) & routes["dst_id"].isin(ids)]

idx = india.set_index("id")[["iata","city","lat","lon"]]

rows = []
for _, r in routes.iterrows():
    try:
        s, d = idx.loc[r["src_id"]], idx.loc[r["dst_id"]]
        dist = haversine(s.lat, s.lon, d.lat, d.lon)
        rows.append({"route": f"{s.iata}-{d.iata}", "distance": dist})
    except:
        pass

df_routes = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

# ---------------- ATF (REAL) ----------------
try:
    end = datetime.today()
    start = datetime(end.year - 5, end.month, end.day)
    fuel = DataReader(ASSUMPTIONS["fred_series"], "fred", start, end).dropna()
    fuel["inr_l"] = (fuel.iloc[:,0] / 3.78541) * ASSUMPTIONS["usd_to_inr"]
    atf_price = fuel["inr_l"].iloc[-1]
except:
    atf_price = 120  # fallback

# ---------------- BASE ECONOMICS ----------------
df = df_routes.copy()
df["pax"] = ASSUMPTIONS["avg_seats"] * ASSUMPTIONS["load_factor"]
df["revenue"] = df["pax"] * (ASSUMPTIONS["avg_fare"] + ASSUMPTIONS["ancillary"])
df["fuel_l"] = df["distance"] * ASSUMPTIONS["avg_seats"] * ASSUMPTIONS["fuel_liters_km_seat"]
df["fuel_cost"] = df["fuel_l"] * atf_price
df["cost"] = df["fuel_cost"] + ASSUMPTIONS["fixed_cost"]
df["margin"] = df["revenue"] - df["cost"]

# SOCIAL WEIGHTS (C)
df["weight"] = 1 + (df["distance"] / df["distance"].max())  # proxy

# ---------------- OPTIMIZATION ENGINE ----------------
results = []

for shock in ASSUMPTIONS["fuel_shocks"]:
    df["margin_shock"] = df["revenue"] - (df["fuel_l"] * atf_price * (1+shock) + ASSUMPTIONS["fixed_cost"])
    df["loss"] = df["margin_shock"].clip(upper=0).abs()

    annual_loss = df["loss"] * ASSUMPTIONS["flights_per_month"] * ASSUMPTIONS["horizon_months"]

    for budget in ASSUMPTIONS["budgets"]:
        prob = pulp.LpProblem("UDAN", pulp.LpMaximize)
        x = pulp.LpVariable.dicts("subsidy", df.index, lowBound=0)

        # A+B+C Objective: weighted margin preserved
        prob += pulp.lpSum(df.loc[i,"weight"] * x[i] for i in df.index)

        # Budget constraint
        prob += pulp.lpSum(x[i] for i in df.index) <= budget

        # Cannot subsidize more than loss
        for i in df.index:
            prob += x[i] <= annual_loss[i]

        prob.solve(pulp.PULP_CBC_CMD(msg=False))

        total_saved = sum(pulp.value(x[i]) for i in df.index)
        routes_saved = sum(1 for i in df.index if pulp.value(x[i]) >= annual_loss[i] * 0.95)

        results.append({
            "fuel_shock": shock,
            "budget_mn": budget/1e6,
            "routes_saved": routes_saved,
            "margin_saved_mn": total_saved/1e6
        })

# ---------------- RESULTS ----------------
res = pd.DataFrame(results)
res.to_csv(f"{OUTDIR}/sensitivity_results.csv", index=False)

# ---------------- PLOTS ----------------
for shock in ASSUMPTIONS["fuel_shocks"]:
    subset = res[res["fuel_shock"] == shock]
    plt.plot(subset["budget_mn"], subset["routes_saved"], marker="o", label=f"{int(shock*100)}% ATF shock")

plt.xlabel("Annual Subsidy Budget (₹ Mn)")
plt.ylabel("Routes Fully Saved")
plt.title("UDAN Impact Sensitivity — MBB Style")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{OUTDIR}/routes_saved_vs_budget.png")
plt.close()

print("DONE. Outputs:")
print(" - outputs/sensitivity_results.csv")
print(" - outputs/routes_saved_vs_budget.png")

DONE. Outputs:
 - outputs/sensitivity_results.csv
 - outputs/routes_saved_vs_budget.png


In [4]:
pip install pandas numpy matplotlib requests pandas_datareader beautifulsoup4 lxml pulp

Note: you may need to restart the kernel to use updated packages.


In [5]:
# ==========================================================
# MBB AIRLINE FUEL CRISIS ENGINE
# Pricing + Subsidy + Exit + Social Weights + Sensitivity
# ==========================================================

import os, io, math, requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pandas_datareader.data import DataReader
from bs4 import BeautifulSoup
import pulp

# ---------------- CONFIG ----------------
OUTDIR = "outputs"
os.makedirs(OUTDIR, exist_ok=True)

CFG = {
    "avg_seats": 78,
    "base_load_factor": 0.72,
    "avg_fare": 3000,
    "ancillary": 400,
    "fixed_cost": 250000,
    "fuel_liters_km_seat": 0.03,
    "usd_to_inr": 82,

    "price_increase": 0.06,
    "elasticity": -1.1,

    "flights_per_month": 60,
    "horizon_months": 12,

    "fuel_shocks": [0.15, 0.25, 0.40],
    "budgets": [5e6, 10e6, 20e6, 40e6],

    "exit_loss_threshold_annual": 2_000_000,  # ₹2M annual loss

    "fred_series": "DJFUELUSGULF",
    "airports_url": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",
    "routes_url": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat",
}

# ---------------- HELPERS ----------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dp = math.radians(lat2 - lat1)
    dl = math.radians(lon2 - lon1)
    a = math.sin(dp/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dl/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# ---------------- ROUTE UNIVERSE ----------------
airports = pd.read_csv(CFG["airports_url"], header=None)
routes = pd.read_csv(CFG["routes_url"], header=None)

airports.columns = ["id","name","city","country","iata","icao","lat","lon","alt","tz","dst","tzdb","type","src"]
routes.columns = ["airline","airline_id","src","src_id","dst","dst_id","codeshare","stops","equip"]

airports["lat"] = pd.to_numeric(airports["lat"], errors="coerce")
airports["lon"] = pd.to_numeric(airports["lon"], errors="coerce")
airports["id"] = pd.to_numeric(airports["id"], errors="coerce")
routes["src_id"] = pd.to_numeric(routes["src_id"], errors="coerce")
routes["dst_id"] = pd.to_numeric(routes["dst_id"], errors="coerce")

india = airports[airports["country"].str.lower() == "india"]
ids = set(india["id"].dropna().astype(int))
routes = routes[routes["src_id"].isin(ids) & routes["dst_id"].isin(ids)]

idx = india.set_index("id")[["iata","city","lat","lon"]]

rows = []
for _, r in routes.iterrows():
    try:
        s, d = idx.loc[r["src_id"]], idx.loc[r["dst_id"]]
        rows.append({
            "route": f"{s.iata}-{d.iata}",
            "distance": haversine(s.lat, s.lon, d.lat, d.lon)
        })
    except:
        pass

df = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

# ---------------- REAL ATF ----------------
try:
    end = datetime.today()
    start = datetime(end.year - 5, end.month, end.day)
    fuel = DataReader(CFG["fred_series"], "fred", start, end).dropna()
    atf = (fuel.iloc[-1,0] / 3.78541) * CFG["usd_to_inr"]
except:
    atf = 120  # fallback INR/L

# ---------------- BASE ECONOMICS ----------------
df["pax"] = CFG["avg_seats"] * CFG["base_load_factor"]
df["revenue"] = df["pax"] * (CFG["avg_fare"] + CFG["ancillary"])
df["fuel_l"] = df["distance"] * CFG["avg_seats"] * CFG["fuel_liters_km_seat"]
df["fuel_cost"] = df["fuel_l"] * atf
df["cost"] = df["fuel_cost"] + CFG["fixed_cost"]
df["margin_base"] = df["revenue"] - df["cost"]

# ---------------- SOCIAL WEIGHTS (REALISTIC PROXY) ----------------
# Distance-weighted connectivity proxy (can be replaced with Census/GDP CSV)
df["social_weight"] = 1 + (df["distance"] / df["distance"].max())

# ---------------- CORE ENGINE ----------------
results = []
route_decisions = []

for shock in CFG["fuel_shocks"]:
    df["fuel_cost_shock"] = df["fuel_l"] * atf * (1 + shock)
    df["cost_shock"] = df["fuel_cost_shock"] + CFG["fixed_cost"]

    # Pricing effect
    df["fare_new"] = CFG["avg_fare"] * (1 + CFG["price_increase"])
    df["lf_new"] = CFG["base_load_factor"] * (1 + CFG["elasticity"] * CFG["price_increase"])
    df["lf_new"] = df["lf_new"].clip(0.4, 0.95)

    df["pax_new"] = CFG["avg_seats"] * df["lf_new"]
    df["revenue_priced"] = df["pax_new"] * (df["fare_new"] + CFG["ancillary"])
    df["margin_priced"] = df["revenue_priced"] - df["cost_shock"]

    df["residual_loss"] = df["margin_priced"].clip(upper=0).abs()
    df["annual_loss"] = df["residual_loss"] * CFG["flights_per_month"] * CFG["horizon_months"]

    for budget in CFG["budgets"]:
        prob = pulp.LpProblem("UDAN_OPT", pulp.LpMaximize)
        x = pulp.LpVariable.dicts("subsidy", df.index, lowBound=0)

        prob += pulp.lpSum(df.loc[i,"social_weight"] * x[i] for i in df.index)
        prob += pulp.lpSum(x[i] for i in df.index) <= budget

        for i in df.index:
            prob += x[i] <= df.loc[i,"annual_loss"]

        prob.solve(pulp.PULP_CBC_CMD(msg=False))

        saved = 0
        exited = 0
        for i in df.index:
            subsidized = pulp.value(x[i])
            if subsidized >= df.loc[i,"annual_loss"] * 0.95:
                saved += 1
            elif df.loc[i,"annual_loss"] > CFG["exit_loss_threshold_annual"]:
                exited += 1

        results.append({
            "fuel_shock": shock,
            "budget_mn": budget/1e6,
            "routes_saved": saved,
            "routes_exit": exited
        })

res = pd.DataFrame(results)
res.to_csv(f"{OUTDIR}/policy_sensitivity.csv", index=False)

# ---------------- VISUAL ----------------
for shock in CFG["fuel_shocks"]:
    sub = res[res["fuel_shock"] == shock]
    plt.plot(sub["budget_mn"], sub["routes_saved"], marker="o", label=f"{int(shock*100)}% ATF")

plt.xlabel("Annual Subsidy Budget (₹ Mn)")
plt.ylabel("Routes Saved")
plt.title("Policy Frontier: Budget vs Routes Saved")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{OUTDIR}/policy_frontier.png")
plt.close()

print("DONE.")
print("Outputs:")
print(" - outputs/policy_sensitivity.csv")
print(" - outputs/policy_frontier.png")

DONE.
Outputs:
 - outputs/policy_sensitivity.csv
 - outputs/policy_frontier.png


In [6]:
# ==========================================================
# MBB AIRLINE POLICY ENGINE — FINAL, HARDENED VERSION
# Pricing + Frequency + Subsidy + Exit + Sensitivity
# NO DASHBOARDS | NO BROKEN URL DEPENDENCIES
# ==========================================================

import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pandas_datareader.data import DataReader
import pulp

# ================= CONFIG =================
OUTDIR = "outputs"
os.makedirs(OUTDIR, exist_ok=True)

CFG = {
    # Fleet & demand
    "avg_seats": 78,
    "base_lf": 0.72,
    "avg_fare": 3000,
    "ancillary": 400,

    # Costs
    "fixed_cost": 250000,
    "fuel_liters_km_seat": 0.03,
    "usd_to_inr": 82,

    # Pricing lever
    "price_increase": 0.06,
    "elasticity": -1.1,

    # Frequency lever
    "freq_reduction": 0.20,        # 20% cut
    "freq_demand_penalty": 0.08,   # demand impact

    # Planning
    "flights_per_month": 60,
    "horizon_months": 12,

    # Policy space
    "fuel_shocks": [0.15, 0.25, 0.40],
    "budgets": [5e6, 10e6, 20e6, 40e6],
    "exit_loss_threshold": 2_000_000,  # INR / year

    # Stable data sources
    "fred_series": "DJFUELUSGULF",
    "airports_url": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",
    "routes_url": "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat",
}

# ================= HELPERS =================
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dp = math.radians(lat2 - lat1)
    dl = math.radians(lon2 - lon1)
    a = math.sin(dp/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dl/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# ================= ROUTE UNIVERSE =================
print("Loading route universe...")

airports = pd.read_csv(CFG["airports_url"], header=None)
routes = pd.read_csv(CFG["routes_url"], header=None)

airports.columns = ["id","name","city","country","iata","icao","lat","lon","alt","tz","dst","tzdb","type","src"]
routes.columns = ["airline","airline_id","src","src_id","dst","dst_id","codeshare","stops","equip"]

airports["lat"] = pd.to_numeric(airports["lat"], errors="coerce")
airports["lon"] = pd.to_numeric(airports["lon"], errors="coerce")
airports["id"] = pd.to_numeric(airports["id"], errors="coerce")
routes["src_id"] = pd.to_numeric(routes["src_id"], errors="coerce")
routes["dst_id"] = pd.to_numeric(routes["dst_id"], errors="coerce")

india = airports[airports["country"].str.lower() == "india"]
ids = set(india["id"].dropna().astype(int))
routes = routes[routes["src_id"].isin(ids) & routes["dst_id"].isin(ids)]

idx = india.set_index("id")[["iata","city","lat","lon"]]

rows = []
for _, r in routes.iterrows():
    try:
        s, d = idx.loc[r["src_id"]], idx.loc[r["dst_id"]]
        rows.append({
            "route": f"{s.iata}-{d.iata}",
            "distance": haversine(s.lat, s.lon, d.lat, d.lon)
        })
    except:
        pass

df = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

# ================= ATF PRICE =================
print("Fetching ATF price (stable FRED fallback)...")

try:
    end = datetime.today()
    start = datetime(end.year - 5, end.month, end.day)
    fuel = DataReader(CFG["fred_series"], "fred", start, end).dropna()
    atf = (fuel.iloc[-1,0] / 3.78541) * CFG["usd_to_inr"]
except:
    atf = 120  # conservative fallback

# ================= SOCIAL WEIGHTS =================
# Robust, dependency-free proxy
df["social_weight"] = 1 + (df["distance"] / df["distance"].max())

# ================= BASE ECONOMICS =================
df["pax"] = CFG["avg_seats"] * CFG["base_lf"]
df["revenue"] = df["pax"] * (CFG["avg_fare"] + CFG["ancillary"])
df["fuel_l"] = df["distance"] * CFG["avg_seats"] * CFG["fuel_liters_km_seat"]
df["fuel_cost"] = df["fuel_l"] * atf
df["cost"] = df["fuel_cost"] + CFG["fixed_cost"]
df["margin_base"] = df["revenue"] - df["cost"]

# ================= CORE ENGINE =================
print("Running policy optimization...")

records = []

for shock in CFG["fuel_shocks"]:
    df["fuel_cost_shock"] = df["fuel_l"] * atf * (1 + shock)

    # Pricing
    df["fare_new"] = CFG["avg_fare"] * (1 + CFG["price_increase"])
    df["lf_priced"] = CFG["base_lf"] * (1 + CFG["elasticity"] * CFG["price_increase"])

    # Frequency rationalisation
    df["lf_final"] = (df["lf_priced"] * (1 - CFG["freq_demand_penalty"])).clip(0.4, 0.95)

    df["pax_final"] = CFG["avg_seats"] * df["lf_final"]
    df["revenue_final"] = df["pax_final"] * (df["fare_new"] + CFG["ancillary"])

    df["cost_final"] = (
        df["fuel_cost_shock"] * (1 - CFG["freq_reduction"]) +
        CFG["fixed_cost"] * (1 - CFG["freq_reduction"])
    )

    df["margin_final"] = df["revenue_final"] - df["cost_final"]
    df["annual_loss"] = df["margin_final"].clip(upper=0).abs() * CFG["flights_per_month"] * CFG["horizon_months"]

    for budget in CFG["budgets"]:
        prob = pulp.LpProblem("POLICY", pulp.LpMaximize)
        x = pulp.LpVariable.dicts("subsidy", df.index, lowBound=0)

        # Objective: maximize weighted impact
        prob += pulp.lpSum(df.loc[i,"social_weight"] * x[i] for i in df.index)

        # Budget
        prob += pulp.lpSum(x[i] for i in df.index) <= budget

        # Cannot subsidize beyond loss
        for i in df.index:
            prob += x[i] <= df.loc[i,"annual_loss"]

        prob.solve(pulp.PULP_CBC_CMD(msg=False))

        saved = sum(
            1 for i in df.index
            if pulp.value(x[i]) >= df.loc[i,"annual_loss"] * 0.95
        )

        exited = sum(
            1 for i in df.index
            if df.loc[i,"annual_loss"] > CFG["exit_loss_threshold"]
            and pulp.value(x[i]) < df.loc[i,"annual_loss"] * 0.5
        )

        records.append({
            "fuel_shock": shock,
            "budget_mn": budget / 1e6,
            "routes_saved": saved,
            "routes_exit": exited
        })

# ================= OUTPUTS =================
res = pd.DataFrame(records)
res.to_csv(f"{OUTDIR}/policy_results_final.csv", index=False)

for shock in CFG["fuel_shocks"]:
    s = res[res["fuel_shock"] == shock]
    plt.plot(s["budget_mn"], s["routes_saved"], marker="o", label=f"{int(shock*100)}% ATF")

plt.xlabel("Annual Subsidy Budget (₹ Mn)")
plt.ylabel("Routes Saved")
plt.title("Policy Frontier: Budget vs Routes Saved")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{OUTDIR}/policy_frontier_final.png")
plt.close()

print("MODEL COMPLETE.")
print("Generated:")
print(" - outputs/policy_results_final.csv")
print(" - outputs/policy_frontier_final.png")


Loading route universe...
Fetching ATF price (stable FRED fallback)...
Running policy optimization...
MODEL COMPLETE.
Generated:
 - outputs/policy_results_final.csv
 - outputs/policy_frontier_final.png
